<a href="https://colab.research.google.com/github/StetskoSergey/pida5_stetsko/blob/master/aml%5Caml%20dz%205.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Что делать?
1.Датасет ml-latest
2.Вспомнить подходы, которые мы разбирали
3.Выбрать понравившийся подход к гибридным системам
4.Написать свою
Материалы здесь

***Сделаем Стэкинг : ***

---


Сначала по жанрам сделаем KNNWithMeans
Сделаем SVD

И Сделаем Ансамбль RandomForestRegressor


In [1]:
pip install surprise

     |████████████████████████████████| 6.5MB 3.0MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678239 sha256=545443143d5b55cec1b1e030be4266e81269ab0db1359018905f9613ab2cdae6
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [0]:
import numpy as np
import pandas as pd

from datetime import datetime

from tqdm import tqdm_notebook

import matplotlib.pyplot as plt
from surprise import Dataset, Reader, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore
from surprise.model_selection import train_test_split, KFold
from sklearn.model_selection import train_test_split as lr_train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from surprise import SVD, SVDpp
from surprise import accuracy
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [0]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [0]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [0]:
# формируем tfidf по жанрам
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

movie_genres = [change_string(g) for g in movies_with_ratings.genres.values]

tfidf_vectorizer_genre = TfidfVectorizer()
X_train_tfidf_genre = tfidf_vectorizer_genre.fit_transform(movie_genres)

In [0]:
# Все сливаем в одну матрицу
genre = X_train_tfidf_genre.toarray()

for x in range(genre.shape[1]):
    col_name = 'g{}'.format(x)
    movies_with_ratings[col_name] = pd.Series(genre[:, x])


In [0]:
dataset = movies_with_ratings.drop(['title','genres','timestamp'], axis = 1)
dataset.dropna(inplace=True)

In [42]:
dataset.shape

(237418, 23)

In [0]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(dataset[['movieId','userId','rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2)

In [0]:
# обучаем модель SVD
algoSVD = SVD(n_factors=20, n_epochs=20)


In [0]:
algoKNNWithMeans = KNNWithMeans(k=25, sim_options={'name':'pearson_baseline', 'user_based':False, 'shrinkage':250})


In [0]:

train, test = lr_train_test_split(dataset, test_size =0.2)
x_train = train.drop(['rating','movieId'], axis = 1)
y_train = train['rating']
x_test = test.drop(['rating','movieId'], axis = 1)
y_test = test['rating']

algoLR = LinearRegression()


In [74]:
x_test.head()

,userId,g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,g13,g14,g15,g16,g17,g18,g19
18303,1041.0,0.514465,0.574429,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.636676,0.0,0.0,0.0
233082,1117.0,0.000000,0.000000,0.0,0.0,0.496655,0.0,0.0,0.000000,0.000000,0.0,0.867948,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.0
181069,48.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.0
69217,2037.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.391014,0.707031,0.0,0.000000,0.0,0.0,0.000000,0.0,0.58925,0.000000,0.0,0.0,0.0
135973,880.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.454517,0.000000,0.0,0.000000,0.0,0.0,0.890738,0.0,0.00000,0.000000,0.0,0.0,0.0


In [47]:
dataset.shape

(237418, 23)

In [0]:
def show_auc(y, y_pred, plot_label='', prin=True):    
    auc_val = 1 - mean_squared_error(y, y_pred, squared=False)/ np.mean(y)
    print('Процент точности предсказания по средней квадратичной ошибке в моделе {0:} : {1:.4f}'.format(plot_label,auc_val))

In [0]:
def get_meta_features(clf, X_train, y_train, X_test):
    #meta_train = np.zeros_like(y_train, dtype=float)
    #meta_test = np.zeros_like(y_test, dtype=float)

    if y_train is None:      
      clf.fit(X_train)
      meta_train = np.array([i.est for i in clf.test(trainset.build_testset())])
      meta_test = np.array([i.est for i in clf.test(X_test)])
    else:
      clf.fit(X_train, y_train)  
      meta_train = np.array(clf.predict(X_train))
      meta_test = np.array(clf.predict(X_test))
    
    return meta_train, meta_test

In [60]:
meta_train = []
meta_test = []
col_names = []

print('algoLR features...')
meta_tr, meta_te = get_meta_features(algoLR, x_train, y_train, x_test)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('algoLR_pred')

print('SVD features...')
meta_tr, meta_te = get_meta_features(algoSVD, trainset, None ,testset)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('algoSVD_pred')

print('algoKNNWithMeans features...')
meta_tr, meta_te = get_meta_features(algoKNNWithMeans, trainset, None, testset)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('algoKNNWithMeans_pred')

algoLR features...
SVD features...
algoKNNWithMeans features...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [69]:
movies_with_genres.head()

,movieId,g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,g13,g14,g15,g16,g17,g18,g19
0,1,0.0,0.362479,0.549991,0.509808,0.293471,0.0,0.0,0.000000,0.469136,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,2,0.0,0.463581,0.000000,0.652003,0.000000,0.0,0.0,0.000000,0.599987,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,3,0.0,0.000000,0.000000,0.000000,0.600305,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.799771,0.0,0.0,0.0,0.0
3,4,0.0,0.000000,0.000000,0.000000,0.530257,0.0,0.0,0.468784,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.706448,0.0,0.0,0.0,0.0
4,5,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [63]:
X_meta_train = pd.DataFrame(np.stack(meta_train, axis=1), columns=col_names)
X_meta_test = pd.DataFrame(np.stack(meta_test, axis=1), columns=col_names)
from sklearn.ensemble import RandomForestRegressor

clf_lr_meta = RandomForestRegressor()
clf_lr_meta.fit(X_meta_train, y_train)
y_pred_meta_test = clf_lr_meta.predict(X_meta_test)
show_auc(y_test, y_pred_meta_test, 'stack')

Процент точности предсказания по средней квадратичной ошибке в моделе stack : 0.6888


In [0]:
movies_with_genres = movies.drop(['title','genres'],  axis = 1)
movie_genres = [change_string(g) for g in movies.genres.values]
genre = tfidf_vectorizer_genre.transform(movie_genres).toarray()

for x in range(genre.shape[1]):
    col_name = 'g{}'.format(x)
    movies_with_genres[col_name] = pd.Series(genre[:, x])

In [0]:
def recommend_for_user(user_id):
    current_user_id = user_id
    movies_with_genres['userId'] = current_user_id
    movies_LR = movies_with_genres.drop('movieId', axis = 1)
        
    meta_ = []
    #titles = []
    LR_predict = algoLR.predict(movies_LR)
    SVD_predict = np.array([algoSVD.predict(uid=current_user_id, iid=i).est for i in movies_with_genres.movieId.values])
    KNNWithMeans_predict = np.array([algoKNNWithMeans.predict(uid=current_user_id, iid=i).est for i in movies_with_genres.movieId.values])
    meta_.append(LR_predict)
    meta_.append(SVD_predict)
    meta_.append(KNNWithMeans_predict)
    
    scores = clf_lr_meta.predict(pd.DataFrame(np.stack(meta_, axis=1), columns=col_names))
   
    
    best_indexes = np.argsort(scores)[-10:]
    for i in reversed(best_indexes):
        print(movies.iloc[[i]].title.values, scores[i])

In [98]:
recommend_for_user(50)

['Children of the Revolution (1996)'] 4.09
['Scream 2 (1997)'] 4.045
['Faces (1968)'] 4.02
['Pompatus of Love, The (1996)'] 4.02
['Everest (1998)'] 4.02
['Tales from the Darkside: The Movie (1990)'] 4.02
['Spice World (1997)'] 4.02
['Girls Town (1996)'] 4.02
['Apt Pupil (1998)'] 4.02
['Metropolis (1927)'] 4.02
